# Robustness Test: Testing Whether FICO Score is Independent of Default Rate

### Section 7.1 Test 3

In a previous test, it was found that loan Grade is probabilistically certain to have a dependent relationship with default rate. In this test, we test the robustness of that by exchanging the variable Grade for FICO score, a similar metric. Everything is basically the same except with FICO score band instead of Grade.

$H_0: P(Default \space|\space FICO \space Score) = P(Default)$

$H_1: P(Default\space |\space FICO \space Score) \neq P(Default)$

We again use the chi square statistic to test independence.

$\chi^2 \sim \sum \frac{(O-E)^2}{E}$


In [1]:
import pandas as pd
import numpy as np
from main import preprocess_df
from collections import defaultdict as dd


df = pd.read_csv(
    "./datasets/lc_data_2007_to_2018.csv",
    low_memory=False,
    encoding="latin1",
    nrows=100000,  # only looking at 100k rows right now for performance
)
pd.set_option("display.max_columns", None)
cleaned_df = preprocess_df(df)

In [2]:
print(cleaned_df["did_default"].value_counts())
overall_percentage_of_default = sum(cleaned_df["did_default"]) / len(cleaned_df) * 100
print(f"Overall percentage of default: {round(overall_percentage_of_default, 2)}%")

did_default
False    70288
True     17603
Name: count, dtype: int64
Overall percentage of default: 20.03%


Again, since 20% of loans in the first 100,000 rows of the dataset were defaulted on, then under $H_0$ we expect each FICO band to have a 20% default rate.


In [ ]:
fico_bands = np.array(sorted([650, 690, 730, 770, 810, 850]))

df_for_testing = cleaned_df.copy()

fico_indices = np.searchsorted(
    fico_bands, df_for_testing["avg_fico"], side="right"
)  # gives a list of each loan's fico band index
fico_indices = np.clip(fico_indices, 0, len(fico_bands) - 1)
df_for_testing.loc[:, "fico_band_index"] = fico_indices


counts_series = df_for_testing["fico_band_index"].value_counts()
loans_per_grade_dict = counts_series.to_dict()
print(loans_per_grade_dict)

{1: 44545, 2: 31633, 3: 8463, 4: 2737, 5: 513}


With FICO bands allotted as follows:

```
Band 0|       score < 650
Band 1| 650 < score < 689
Band 2| 690 < score < 729
Band 3| 730 < score < 769
Band 4| 770 < score < 809
Band 5| 810 < score < 850
```

The total count of loans in each FICO band are displayed in this table.

```
0 |     0
1 | 44545
2 | 31633
3 |  8463
4 |  2737
5 |   513
```


In [ ]:
expected_defaults_dict = dd(int)
fico_band_indices = set(fico_indices)  # to get only the uniques

for band_index in fico_band_indices:
    specific_band_df = df_for_testing.loc[
        df_for_testing["fico_band_index"] == band_index, :
    ]
    expected_num_defaults = 0.2 * len(specific_band_df)
    expected_defaults_dict[band_index] = expected_num_defaults
    # find the observed number of defaults for each grade
    print(f"{band_index} | {round(expected_num_defaults)}")

1 | 8909
2 | 6327
3 | 1693
4 | 547
5 | 103


Since $H_0$ states that FICO band and Default Rate are independent, we should expect all FICO bands to have equal rates of default.

Therefore, since overall rate of default is 20%, we should expect 20% of all loans in each FICO band to be defaulted:

### Expected Default Counts for Each FICO band

```
1 | 8909
2 | 6327
3 | 1693
4 |  547
5 |  103
```


In [ ]:
# find the observed number of defaults for each fico band

observed_defaults_dict = dd(int)
for band_index in fico_band_indices:
    specific_band_df = df_for_testing.loc[
        df_for_testing["fico_band_index"] == band_index, :
    ]
    observed_num_defaults = sum(specific_band_df["did_default"])
    observed_defaults_dict[band_index] = observed_num_defaults
    print(f"{band_index} | {round(observed_num_defaults)}")

1 | 10874
2 | 5583
3 | 933
4 | 191
5 | 22


### Observed Default Counts for Each FICO band

```
1 | 10874
2 |  5583
3 |   933
4 |   191
5 |    22
```


In [ ]:
for band_index in fico_band_indices:
    num_loans_in_band = sum(df_for_testing["fico_band_index"] == band_index)
    num_defaulted_loans_in_band = len(
        df_for_testing[
            (df_for_testing["fico_band_index"] == band_index)
            & (df_for_testing["did_default"])
        ]
    )
    percentage_of_defaults = num_defaulted_loans_in_band / num_loans_in_band * 100
    print(f"{band_index} | {round(percentage_of_defaults, 2)}")

1 | 24.41
2 | 17.65
3 | 11.02
4 | 6.98
5 | 4.29


### Observed percentage rates of Default

Some FICO bands had a much smaller sample size than others, band 5 being the smallest at $n=513$. Different sample sizes make it difficult to rely on counts to see trends. For illustrative purposes, the percentage rates of default for each band are displayed in this table.

```
1 | 24.41%
2 | 17.65%
3 | 11.02%
4 |  6.98%
5 |  4.29%
```


In [ ]:
from scipy.stats import chi2
import math

print(observed_defaults_dict)
print(expected_defaults_dict)

default_scaled_sqd_devs = [
    (observed_defaults_dict[band_index] - expected_defaults_dict[band_index]) ** 2
    / (expected_defaults_dict[band_index])
    for band_index in fico_band_indices
]
non_default_scaled_sqd_devs = [
    (observed_defaults_dict[band_index] - expected_defaults_dict[band_index]) ** 2
    / (loans_per_grade_dict[band_index] - expected_defaults_dict[band_index])
    for band_index in fico_band_indices
]
test_stat = sum(default_scaled_sqd_devs) + sum(non_default_scaled_sqd_devs)
print(default_scaled_sqd_devs)
print(non_default_scaled_sqd_devs)

print("test_stat:", test_stat)

chi2_stat = test_stat
df = 5

log_p = chi2.logsf(chi2_stat, df)
print("log_p:", log_p)

p_value = math.exp(log_p)

print("p_value:", p_value)

defaultdict(<class 'int'>, {np.int64(1): 10874, np.int64(2): 5583, np.int64(3): 933, np.int64(4): 191, np.int64(5): 22})
defaultdict(<class 'int'>, {np.int64(1): 8909.0, np.int64(2): 6326.6, np.int64(3): 1692.6000000000001, np.int64(4): 547.4, np.int64(5): 102.60000000000001})
[433.40722864519023, 87.39938671640384, 340.89103154909617, 232.04413591523561, 63.31734892787525]
[108.35180716129756, 21.84984667910096, 85.22275788727406, 58.011033978808904, 15.829337231968815]
test_stat: 1446.3239146922513
log_p: -713.5691169037973
p_value: 1.26145009006517e-310


### The Test

For our test statistic, we must compute $\chi^2 \sim \sum \frac{(O-E)^2}{E}$ over all cells. That means not only the expected/observed defaults, but the expected/observed non-defaults as well.

Our test statistic comes out to be $1446.32$.

Since we are testing FICO bands 0-5, we have 6 bands. And we are testing their impact on a binary variable, did default or did not default. Therefore the contingency table is 2x6. And degrees of freedom are calculated by (c-1)(r-1), or in this case (1)(5) = 5. So there are 5 degrees of freedom.

The probability of getting a value equal to or larger (more extreme, since chi-square is right-tailed) than $1446.32$ on a chi-square distribution with 5 df, also known as the $p$-value, is $1.26\times 10^{-310}$, so we reject $H_0$.
